In [6]:
import os
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime
from sqlalchemy import create_engine

In [ ]:
# Declaração das pastas

pasta_itau = r"desktop\Extrato\Itaú"
pasta_Bradesco = r"desktop\Extrato\Bradesco"
pasta_Safra = r"desktop\Extrato\Safra"
pasta_XP = r"desktop\Extrato\XP"
Pasta_BancoDoBrasil = r"desktop\Extrato\Banco do Brasil"

In [9]:
# Varíaveis de todos os métodos

Colunas = ["Filial", "Banco", "Data", "Lancamento", "DebitoCredito", "Valor", "Saldo"]

# Colocar o Ano na data
ano_atual = datetime.now().year


def adicionar_ano(data_str):
    if pd.isna(data_str):  # Verificar se o valor é NaN
        return None  # Ou outro valor de sua escolha
    else:
        # Garantir que a string tenha pelo menos dois elementos (dia e mês)
        if "/" in data_str:
            partes = data_str.split("/")
            if (
                len(partes) == 2
            ):  # Verificar se a divisão resultou em 2 partes (dia e mês)
                dia, mes = partes
                nova_data = f"{dia}/{mes}/{ano_atual}"
                return nova_data
            else:
                return None  # Retornar None ou outro valor se o formato não for válido
        else:
            return None  # Retornar None ou outro valor se não houver '/'


data_hoje = datetime.today().strftime("%Y-%m-%d")


# Deletar arquivos diretamente
# Deletar arquivos de origem


def deletar_arquivos(caminho):
    try:
        # Verificar se o arquivo existe
        if os.path.isfile(caminho):
            os.remove(caminho)  # Deleta o arquivo
            print(f"Arquivo {caminho} deletado com sucesso.")
        else:
            print(f"O caminho {caminho} não é um arquivo válido.")
    except FileNotFoundError:
        print(f"Arquivo {caminho} não encontrado.")
    except Exception as e:
        print(f"Erro ao tentar deletar o arquivo {caminho}: {e}")

In [10]:
# INÍCIO MÉTODO ITAÚ

try:
    lista_arquivos_itau = os.listdir(pasta_itau)
    if lista_arquivos_itau:
        primeiro_arquivo_itau = lista_arquivos_itau[0]
        caminho_itau = os.path.join(pasta_itau, primeiro_arquivo_itau)
        print(f"Lendo o arquivo: {caminho_itau}")
    else:
        print(f"A pasta {pasta_itau} está vazia.")
except FileNotFoundError:
    print(f"Diretório não encontrado: {pasta_itau}")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

A pasta Q:\COMP_PF\Arquivos Vinicius\Extrato\Itaú está vazia.


In [11]:
# Ajuste de escopo

# Início e fim das linhas
df_itau = pd.read_html(caminho_itau, header=0, skiprows=10)[0]
df_itau = df_itau.dropna(axis=1, how="all")
df_itau_list = pd.read_html(caminho_itau, header=0)
# filtro_itau = df_itau[df_itau['Lançamento'] == 'S A L D O'].index[0]
# df_itau = df_itau.iloc[:filtro_itau + 1]
df_itau = df_itau[df_itau["Lançamento"] != "S A L D O"]
df_itau = df_itau.iloc[1:]

# Tipos de campos
df_itau["Data"] = df_itau["Data"].astype(str)
df_itau["Valor (R$)"] = df_itau["Valor (R$)"].fillna(0)
df_itau["Saldo (R$)"] = 0

NameError: name 'caminho_itau' is not defined

In [ ]:
# Inserir colunas adicionais
df_itau["Data Completa"] = df_itau["Data"].apply(adicionar_ano)
df_itau["Banco"] = "Itau"
df_itau["DebitoCredito"] = df_itau["Valor (R$)"].apply(
    lambda x: 1 if x < 0 else (2 if x > 0 else 0)
)

# Remover a coluna 'Data'
df_itau = df_itau.drop(columns=["Data"])

# Renomear Colunas

df_itau = df_itau.rename(
    columns={
        "Data Completa": "Data",
        "Lançamento": "Lancamento",
        "Saldo (R$)": "Saldo",
        "Valor (R$)": "Valor",
    }
)

# Ordenar colunas

df_itau = df_itau[Colunas]

In [ ]:
# INÍCIO MÉTODO BRADESCO
try:
    lista_arquivos_Bradesco = os.listdir(pasta_Bradesco)
    if lista_arquivos_Bradesco:
        primeiro_arquivo_Bradesco = lista_arquivos_Bradesco[0]
except FileNotFoundError:
    print(f"Diretório não encontrado: {pasta_Bradesco}")

'try:\n    lista_arquivos_Bradesco = os.listdir(pasta_Bradesco)\n    if lista_arquivos_Bradesco:\n        primeiro_arquivo_Bradesco = lista_arquivos_Bradesco[0]\nexcept FileNotFoundError:\n    print(f"Diretório não encontrado: {pasta_Bradesco}")\n    '

In [ ]:
caminho_Bradesco = os.path.join(pasta_Bradesco, primeiro_arquivo_Bradesco)
print(f"Lendo o arquivo: {caminho_Bradesco}")

'caminho_Bradesco = os.path.join(pasta_Bradesco, primeiro_arquivo_Bradesco)\nprint(f"Lendo o arquivo: {caminho_Bradesco}")'

In [ ]:
# Ajuste de escopo

# Início e fim das linhas
df_Bradesco = pd.read_excel(caminho_Bradesco, header=0, skiprows=8)
filtro_bradesco = df_Bradesco[df_Bradesco['Data'] == 'Total'].index[0]
df_Bradesco = df_Bradesco.iloc[:filtro_bradesco + 1]
df_Bradesco = df_Bradesco[df_Bradesco['Data'] != 'Total']
df_Bradesco = df_Bradesco.iloc[1:]

# Tipos de campos

# Primeiro, substitua os pontos e vírgulas nas strings
df_Bradesco['Crédito (R$)'] = df_Bradesco['Crédito (R$)'].astype(str).str.replace('.', '', regex=False)
df_Bradesco['Crédito (R$)'] = df_Bradesco['Crédito (R$)'].str.replace(',', '.', regex=False)
df_Bradesco['Débito (R$)'] = df_Bradesco['Débito (R$)'].astype(str).str.replace('.', '', regex=False)
df_Bradesco['Débito (R$)'] = df_Bradesco['Débito (R$)'].str.replace(',', '.', regex=False)
df_Bradesco['Saldo (R$)'] = df_Bradesco['Saldo (R$)'].astype(str).str.replace('.', '', regex=False)
df_Bradesco['Saldo (R$)'] = df_Bradesco['Saldo (R$)'].str.replace(',', '.', regex=False)

# Agora, converta as colunas para numéricas
df_Bradesco['Crédito (R$)'] = pd.to_numeric(df_Bradesco['Crédito (R$)'], errors='coerce')
df_Bradesco['Débito (R$)'] = pd.to_numeric(df_Bradesco['Débito (R$)'], errors='coerce')
df_Bradesco['Saldo (R$)'] = pd.to_numeric(df_Bradesco['Saldo (R$)'], errors='coerce')

"import pandas as pd\n\n# Ajuste de escopo\n\n# Início e fim das linhas\ndf_Bradesco = pd.read_excel(caminho_Bradesco, header=0, skiprows=8)\nfiltro_bradesco = df_Bradesco[df_Bradesco['Data'] == 'Total'].index[0]\ndf_Bradesco = df_Bradesco.iloc[:filtro_bradesco + 1]\ndf_Bradesco = df_Bradesco[df_Bradesco['Data'] != 'Total']\ndf_Bradesco = df_Bradesco.iloc[1:]\n\n# Tipos de campos\n\n# Primeiro, substitua os pontos e vírgulas nas strings\ndf_Bradesco['Crédito (R$)'] = df_Bradesco['Crédito (R$)'].astype(str).str.replace('.', '', regex=False)\ndf_Bradesco['Crédito (R$)'] = df_Bradesco['Crédito (R$)'].str.replace(',', '.', regex=False)\ndf_Bradesco['Débito (R$)'] = df_Bradesco['Débito (R$)'].astype(str).str.replace('.', '', regex=False)\ndf_Bradesco['Débito (R$)'] = df_Bradesco['Débito (R$)'].str.replace(',', '.', regex=False)\ndf_Bradesco['Saldo (R$)'] = df_Bradesco['Saldo (R$)'].astype(str).str.replace('.', '', regex=False)\ndf_Bradesco['Saldo (R$)'] = df_Bradesco['Saldo (R$)'].str.repla

In [ ]:
# Inserir colunas adicionais
df_Bradesco['Banco'] = "Bradesco"
df_Bradesco['DebitoCredito'] = df_Bradesco.apply(lambda row: 2 if row['Crédito (R$)'] > 0 else (1 if row['Débito (R$)'] < 0 else 0), axis=1)
df_Bradesco['Valor'] = df_Bradesco['Crédito (R$)'].fillna(0) + df_Bradesco['Débito (R$)'].fillna(0)
df_Bradesco = df_Bradesco.drop(columns=['Dcto.', 'Débito (R$)', 'Crédito (R$)'])

df_Bradesco = df_Bradesco.rename(columns={'Lançamento': 'Lancamento',
'Saldo (R$)': 'Saldo'
})

df_Bradesco = df_Bradesco[Colunas]

# Fim Metodo Bradesco

'df_Bradesco[\'Banco\'] = "Bradesco"\ndf_Bradesco[\'DebitoCredito\'] = df_Bradesco.apply(lambda row: 2 if row[\'Crédito (R$)\'] > 0 else (1 if row[\'Débito (R$)\'] < 0 else 0), axis=1)\ndf_Bradesco[\'Valor\'] = df_Bradesco[\'Crédito (R$)\'].fillna(0) + df_Bradesco[\'Débito (R$)\'].fillna(0)\ndf_Bradesco[\'Filial\'] = atribuir_filial(caminho_Bradesco)\ndf_Bradesco = df_Bradesco.drop(columns=[\'Dcto.\', \'Débito (R$)\', \'Crédito (R$)\'])\n\ndf_Bradesco = df_Bradesco.rename(columns={\'Lançamento\': \'Lancamento\',\n\'Saldo (R$)\': \'Saldo\'\n})\n\ndf_Bradesco = df_Bradesco[Colunas]\n'

In [ ]:
# INÍCIO MÉTODO SAFRA

try:
    lista_arquivos_Safra = os.listdir(pasta_Safra)
    if lista_arquivos_Safra:
        primeiro_arquivo_Safra = lista_arquivos_Safra[0]
except FileNotFoundError:
    print(f"Diretório não encontrado: {pasta_Safra}")
    
    

'\ntry:\n    lista_arquivos_Safra = os.listdir(pasta_Safra)\n    if lista_arquivos_Safra:\n        primeiro_arquivo_Safra = lista_arquivos_Safra[0]\nexcept FileNotFoundError:\n    print(f"Diretório não encontrado: {pasta_Safra}")\n    \n    '

In [ ]:
caminho_Safra = os.path.join(pasta_Safra, primeiro_arquivo_Safra)
print(f"Lendo o arquivo: {caminho_Safra}")

'caminho_Safra = os.path.join(pasta_Safra, primeiro_arquivo_Safra)\nprint(f"Lendo o arquivo: {caminho_Safra}")'

In [ ]:
# Ajuste do escopo

df_Safra = pd.read_html(caminho_Safra, header=None)
df_Safra = df_Safra.iloc[:, :8] 
df_Safra = df_Safra.iloc[12:].reset_index(drop=True)
df_Safra.columns = ['Data', 'Situação', 'Tipo do Lançamento', 'Lançamento', 'Complemento', 'Nº Documento', 'Valor', 'Saldo']

#Início e fim das linhas
df_Safra = df_Safra[~df_Safra['Lançamento'].isin(['SALDO INICIAL', 'SALDO TOTAL'])]
df_Safra = df_Safra.dropna(subset=['Data'])

# Substituir valores vazios (NaN) nas colunas 'Valor' e 'Saldo' por 0
df_Safra['Valor'] = df_Safra['Valor'].fillna(0)
df_Safra['Saldo'] = df_Safra['Saldo'].fillna(0)


"\ndf_Safra = pd.read_html(caminho_Safra, header=None)\ndf_Safra = df_Safra.iloc[:, :8] \ndf_Safra = df_Safra.iloc[12:].reset_index(drop=True)\ndf_Safra.columns = ['Data', 'Situação', 'Tipo do Lançamento', 'Lançamento', 'Complemento', 'Nº Documento', 'Valor', 'Saldo']\n\n#Início e fim das linhas\ndf_Safra = df_Safra[~df_Safra['Lançamento'].isin(['SALDO INICIAL', 'SALDO TOTAL'])]\ndf_Safra = df_Safra.dropna(subset=['Data'])\n\n# Substituir valores vazios (NaN) nas colunas 'Valor' e 'Saldo' por 0\ndf_Safra['Valor'] = df_Safra['Valor'].fillna(0)\ndf_Safra['Saldo'] = df_Safra['Saldo'].fillna(0)\n"

In [ ]:
# Inserir colunas adicionais
df_Safra['Data Completa'] = df_Safra['Data'].apply(adicionar_ano)
df_Safra['Banco'] = "Safra"
df_Safra['DebitoCredito'] = df_Safra['Tipo do Lançamento'].apply(lambda x: 1 if x == 'Débito' else (2 if x == 'Crédito' else 0))


df_Safra = df_Safra.drop(columns=['Data', 'Situação', 'Tipo do Lançamento', 'Nº Documento','Lançamento'])

df_Safra = df_Safra.rename(columns={'Data Completa': 'Data','Complemento': 'Lancamento'})

df_Safra = df_Safra[Colunas]

# Fim do Metodo Safra

'# Inserir colunas adicionais\ndf_Safra[\'Data Completa\'] = df_Safra[\'Data\'].apply(adicionar_ano)\ndf_Safra[\'Banco\'] = "Safra"\ndf_Safra[\'DebitoCredito\'] = df_Safra[\'Tipo do Lançamento\'].apply(lambda x: 1 if x == \'Débito\' else (2 if x == \'Crédito\' else 0))\ndf_Safra[\'Filial\'] = atribuir_filial(caminho_Safra)\n\ndf_Safra = df_Safra.drop(columns=[\'Data\', \'Situação\', \'Tipo do Lançamento\', \'Nº Documento\',\'Lançamento\'])\n\ndf_Safra = df_Safra.rename(columns={\'Data Completa\': \'Data\',\'Complemento\': \'Lancamento\'})\n\ndf_Safra = df_Safra[Colunas]\n\n# Fim do Metodo Safra'

In [ ]:
# INICIO MÉTODO XP

try:
    lista_arquivos_XP = os.listdir(pasta_XP)
    if lista_arquivos_XP:
        primeiro_arquivo_XP = lista_arquivos_XP[0]
except FileNotFoundError:
    print(f"Diretório não encontrado: {pasta_XP}")

'\ntry:\n    lista_arquivos_XP = os.listdir(pasta_XP)\n    if lista_arquivos_XP:\n        primeiro_arquivo_XP = lista_arquivos_XP[0]\nexcept FileNotFoundError:\n    print(f"Diretório não encontrado: {pasta_XP}")'

In [ ]:
caminho_XP = os.path.join(pasta_XP, primeiro_arquivo_XP)
print(f"Lendo o arquivo: {caminho_XP}")

'caminho_XP = os.path.join(pasta_XP, primeiro_arquivo_XP)\nprint(f"Lendo o arquivo: {caminho_XP}")'

In [ ]:
# Ajuste de escopo

# Início e fim das linhas
df_XP = pd.read_excel(caminho_XP, header=0,skiprows=13)


df_XP = df_XP.dropna(axis=1, how='all')
filtro_XP = df_XP[df_XP['Movimentação'] == 'Lançamentos futuros'].index[0]
df_XP = df_XP.iloc[:filtro_XP - 1]
df_XP = df_XP.dropna(axis=1, how='all')

"# Ajuste de escopo\n\n# Início e fim das linhas\ndf_XP = pd.read_excel(caminho_XP, header=0,skiprows=13)\n\n\ndf_XP = df_XP.dropna(axis=1, how='all')\nfiltro_XP = df_XP[df_XP['Movimentação'] == 'Lançamentos futuros'].index[0]\ndf_XP = df_XP.iloc[:filtro_XP - 1]\ndf_XP = df_XP.dropna(axis=1, how='all')"

In [ ]:
# Inserir colunas adicionais

df_XP['Banco'] = "XP"
df_XP['DebitoCredito'] = df_XP['Valor (R$)'].apply(lambda x: 1 if x < 0 else (2 if x > 0 else 0))

# Remover a colunas
df_XP = df_XP.drop(columns=['Movimentação'])

# Renomear Colunas
df_XP = df_XP.rename(columns={
    'Liquidação': 'Data',
    'Lançamento': 'Lancamento',
    'Saldo (R$)': 'Saldo',
    'Valor (R$)': 'Valor'
})
df_XP = df_XP[Colunas]

# Fim Método XP


'# Inserir colunas adicionais\n\ndf_XP[\'Banco\'] = "XP"\ndf_XP[\'DebitoCredito\'] = df_XP[\'Valor (R$)\'].apply(lambda x: 1 if x < 0 else (2 if x > 0 else 0))\ndf_XP[\'Filial\'] = atribuir_filial(caminho_XP)\n\n# Remover a colunas\ndf_XP = df_XP.drop(columns=[\'Movimentação\'])\n\n# Renomear Colunas\ndf_XP = df_XP.rename(columns={\n    \'Liquidação\': \'Data\',\n    \'Lançamento\': \'Lancamento\',\n    \'Saldo (R$)\': \'Saldo\',\n    \'Valor (R$)\': \'Valor\'\n})\ndf_XP = df_XP[Colunas]\n\n# Fim Método XP\n'

In [ ]:
# Inicio Metodo Banco do Brasil
try:
    lista_arquivos_BancoDoBrasil = os.listdir(Pasta_BancoDoBrasil)
    if lista_arquivos_BancoDoBrasil:
        primeiro_arquivo_BancoDoBrasil = lista_arquivos_BancoDoBrasil[0]
except FileNotFoundError:
    print(f"Diretório não encontrado: {Pasta_BancoDoBrasil}")

'try:\n    lista_arquivos_BancoDoBrasil = os.listdir(Pasta_BancoDoBrasil)\n    if lista_arquivos_BancoDoBrasil:\n        primeiro_arquivo_BancoDoBrasil = lista_arquivos_BancoDoBrasil[0]\nexcept FileNotFoundError:\n    print(f"Diretório não encontrado: {Pasta_BancoDoBrasil}")'

In [ ]:
caminho_BancoDoBrasil = os.path.join(Pasta_BancoDoBrasil, primeiro_arquivo_BancoDoBrasil)
print(f"Lendo o arquivo: {caminho_BancoDoBrasil}")

'caminho_BancoDoBrasil = os.path.join(Pasta_BancoDoBrasil, primeiro_arquivo_BancoDoBrasil)\nprint(f"Lendo o arquivo: {caminho_BancoDoBrasil}")'

In [ ]:
# Ajuste de escopo

# Carregar o DataFrame
df_BancoDoBrasil = pd.read_excel(caminho_BancoDoBrasil, header=0, skiprows=2)

# Filtrar as linhas onde 'Lote' não é igual a 0
df_BancoDoBrasil = df_BancoDoBrasil[df_BancoDoBrasil['Lote'] != 0]



"\n# Carregar o DataFrame\ndf_BancoDoBrasil = pd.read_excel(caminho_BancoDoBrasil, header=0, skiprows=2)\n\n# Filtrar as linhas onde 'Lote' não é igual a 0\ndf_BancoDoBrasil = df_BancoDoBrasil[df_BancoDoBrasil['Lote'] != 0]\n\n"

In [ ]:

#Adicionar colunas
df_BancoDoBrasil['Banco'] = "Banco Do Brasil"
df_BancoDoBrasil['DebitoCredito'] = df_BancoDoBrasil['Inf.'].apply(lambda x: 1 if x == 'D' else (2 if x == 'C' else 0))
df_BancoDoBrasil['Saldo'] = 0

# Remover a colunas
df_BancoDoBrasil = df_BancoDoBrasil.drop(columns=['observacao','Data balancete','Agencia Origem','Lote','Numero Documento','Cod. Historico','Inf.','Detalhamento Hist.'])

# Renomear Colunas
df_BancoDoBrasil = df_BancoDoBrasil.rename(columns={
    'Historico': 'Lancamento',
    'Valor R$ ': 'Valor',
})
df_BancoDoBrasil['Valor'] = df_BancoDoBrasil['Valor'].str.replace('.', '', regex=False)
df_BancoDoBrasil['Valor'] = df_BancoDoBrasil['Valor'].str.replace(',', '.', regex=False)
df_BancoDoBrasil['Valor'] = df_BancoDoBrasil['Valor'].astype(float)  # Converte para float
df_BancoDoBrasil = df_BancoDoBrasil[Colunas]



'\n#Adicionar colunas\ndf_BancoDoBrasil[\'Banco\'] = "Banco Do Brasil"\ndf_BancoDoBrasil[\'DebitoCredito\'] = df_BancoDoBrasil[\'Inf.\'].apply(lambda x: 1 if x == \'D\' else (2 if x == \'C\' else 0))\ndf_BancoDoBrasil[\'Filial\'] = atribuir_filial(caminho_BancoDoBrasil)\ndf_BancoDoBrasil[\'Saldo\'] = 0\n\n# Remover a colunas\ndf_BancoDoBrasil = df_BancoDoBrasil.drop(columns=[\'observacao\',\'Data balancete\',\'Agencia Origem\',\'Lote\',\'Numero Documento\',\'Cod. Historico\',\'Inf.\',\'Detalhamento Hist.\'])\n\n# Renomear Colunas\ndf_BancoDoBrasil = df_BancoDoBrasil.rename(columns={\n    \'Historico\': \'Lancamento\',\n    \'Valor R$ \': \'Valor\',\n})\ndf_BancoDoBrasil[\'Valor\'] = df_BancoDoBrasil[\'Valor\'].str.replace(\'.\', \'\', regex=False)\ndf_BancoDoBrasil[\'Valor\'] = df_BancoDoBrasil[\'Valor\'].str.replace(\',\', \'.\', regex=False)\ndf_BancoDoBrasil[\'Valor\'] = df_BancoDoBrasil[\'Valor\'].astype(float)  # Converte para float\ndf_BancoDoBrasil = df_BancoDoBrasil[Colunas]\n\

In [ ]:
# Concatenar os dois DataFrames
# df_completo = pd.concat([df_Bradesco, df_itau,df_Safra,df_XP,df_BancoDoBrasil], ignore_index=True)
df_completo = pd.concat([df_itau], ignore_index=True)

# Aplicar o cabeçalho da variável 'Colunas'
df_completo.columns = Colunas

# Caminho do arquivo de saída
output_file = f"extrato_processado_{data_hoje}.xlsx"

# Salvar o DataFrame concatenado no Excel
df_completo.to_excel(output_file, index=False)

print(f"Arquivo salvo com sucesso em {output_file}")

Arquivo salvo com sucesso em extrato_processado_2025-02-24.xlsx


In [ ]:
# Ajuste de coluna com valor nulo
df_completo["Lancamento"] = df_completo["Lancamento"].apply(
    lambda x: None if pd.isna(x) or x == "" else x
)
df_completo["Lancamento"] = df_completo["Lancamento"].astype(str)
df_completo["data_hora"] = datetime.now()

In [ ]:
# Deletar arquivos base
deletar_arquivos(caminho_Safra)
deletar_arquivos(caminho_Bradesco)
deletar_arquivos(caminho_XP)
deletar_arquivos(caminho_BancoDoBrasil)
deletar_arquivos(caminho_itau)

Arquivo Q:\COMP_PF\Arquivos Vinicius\Extrato\Itaú\extrato 99890SP 01-02 A 11-02 SP.xls deletado com sucesso.
Arquivo Q:\COMP_PF\Arquivos Vinicius\Extrato\Pagamentos\ConsultaPagamentos 99890-4 SP 01-02 A 11-02 SP.xls deletado com sucesso.
